# Memória, Variáveis, Comunicação e Estado de um Programa

Há várias possíveis estratégias para usar a memória de um computador para se realizar a comunicação entre partes dele -- por exemplo, entre duas funções -- e para se manter o **estado atual** de execução de um programa. Dentre elas, podemos destacar o uso de variáveis locais, globais e parâmetros entre funções.

Vamos fazer aqui um exercício nesse contexto e você será conduzido a experimentar cada uma dessas estratégias, mesmo que em algumas ocasiões não seja a forma recomendada de se desenvolver um programa.


# Exercício do Empréstimo

Em um financiamento com juros compostos e número de parcelas fixas parte-se dos seguintes parâmetros:
* `S` - valor da primeira parcela
* `N` - número de parcelas
* `J` - percentual de juros mensal

A primeira parcela a ser paga do financiamento é sempre igual a `S`. A partir daí é feita uma atualização mensal da parcela, em que cada nova parcela é calculada a partir da parcela do mês anterior, conforme a fórmula:

> Parcela<sub>mês</sub> = Parcela<sub>mês-1</sub> * (1 + `J` / 100)

O financiamento encerra quando as `N` parcelas são pagas.

Exemplo:
* `S`: 200
* `N`: 5
* `J`: 1%

Parcelas do financiamento:
`200`; `202`; `204.02`; `206.06`; `208.12`

## Exercício Parte 1 - Escrevendo um Módulo

Dado o problema descrito, escreva um programa que calcule as parcelas de um empréstimo para os seguintes valores:
* `S`: 200
* `N`: 5
* `J`: 1%

In [1]:
#include <stdio.h>

int main() {
    float s = 200;
    int n = 5;
    float j = 1;
    
    float p = s;
    for (int i = 1; i <= n; i++) {
        printf("O valor da parcela %d eh %3.2f\n", i, p);
        p = p + (p * (j/100));
    }   
}

O valor da parcela 1 eh 200.00
O valor da parcela 2 eh 202.00
O valor da parcela 3 eh 204.02
O valor da parcela 4 eh 206.06
O valor da parcela 5 eh 208.12


## Exercício Parte 2 - Escrevendo um Módulo (Função)

Reescreva o código acima de forma que seu programa faça uso de uma função `proximaParcela` que seja responsável pelo cálculo de uma única parcela X do empréstimo. Utilize as boas práticas de modularização que você aprendeu, evitando dependências do módulo com o programa principal. A função deve apenas calcular uma única parcela, portanto, ficará a cargo da função principal que chama `proximaParcela` realizar o restante do processo, para que sejam apresentadas todas as parcelas do financiamento.

In [2]:
#include <stdio.h>

float proximaParcela(float s, int n, float j, int *corrente, float *p) {
    if (*corrente == 0)   // primeira parcela
        *p = s;
    else if (*corrente <= n)
        *p = *p + (*p * (j/100));
    else
        *p = 0;  // não há mais parcelas
    (*corrente)++;
    return *p;
}

int main() {
    float s = 200;
    int n = 5;
    float j = 1;
    int corrente = 0;  // controla parcela corrente
    float p = s;  // controla valor da parcela corrente

    for (int i = 1; i <= n; i++) {
        p = proximaParcela(s, n, j, &corrente, &p);
        printf("O valor da parcela %d eh %3.2f\n", i, p);
    }   
}

O valor da parcela 1 eh 200.00
O valor da parcela 2 eh 202.00
O valor da parcela 3 eh 204.02
O valor da parcela 4 eh 206.06
O valor da parcela 5 eh 208.12


## Exercício Parte 3 - Mantendo o Estado

Modifique a função (módulo) `proximaParcela` acima de modo que ela tenha uma memória da situação em que estão as coisas, sem que o `main` precise ficar informando novamente coisas que já informou no início. Para isso serão permitidas as seguintes modificações:

1. você pode desmembrar a função em mais de uma, por exemplo, uma você chama no começo do empréstimo e outra a cada parcela;

2. você pode usar técnicas de uso de variáveis não recomendadas que geram dependência dos módulos com o programa principal.

Você deve organizar o código de tal maneira que o `main` informe para as funções os dados do financiamento apenas uma única vez e depois possa solicitar o cálculo da parcela subsequente sem informar tudo novamente.

In [3]:
#include <stdio.h>

float es;
int   en;
float ej;
int   corrente;
float p;

void novoEmprestimo(float s, int n, float j) {
    es = s;
    en = n;
    ej = j;
    corrente = 1;
    p = s;
}

float proximaParcela() {
    float retorno = p;
    corrente++;
    if (corrente <= en)
        p = p + (p * (ej/100));
    else
        p = 0;
    return retorno;
}

int main() {
    novoEmprestimo(200, 5, 1);
    
    int i = 1;
    float p = proximaParcela();
    while (p > 0) {
        printf("O valor da parcela %d eh %3.2f\n", i, p);
        p = proximaParcela();
        i++;
    }   
}

O valor da parcela 1 eh 200.00
O valor da parcela 2 eh 202.00
O valor da parcela 3 eh 204.02
O valor da parcela 4 eh 206.06
O valor da parcela 5 eh 208.12


## Exercício Parte 4 - Minimizando os Parâmetros

Retomando a solução da `Parte 2`, em que a função `main` deve ter usado vários parâmetros para a comunicação com a função `proximaParcela`,  modifique a forma como você representa o empréstimo, de forma que a função `proximaParcela` receba sempre um único parâmetro capaz de representar o estado completo do empréstimo.

### Agrupando variáveis de estado em uma estrutura

In [4]:
#include <stdio.h>

typedef struct {
    float s;
    int   n;
    float j;
    int   corrente;
    float p;
} Emprestimo;

void novoEmprestimo(Emprestimo *umEmprestimo) {
    umEmprestimo->corrente = 1;
    umEmprestimo->p = umEmprestimo->s;
}

float proximaParcela(Emprestimo *umEmprestimo) {
    float retorno = umEmprestimo->p;
    if (umEmprestimo->corrente < umEmprestimo->n)
        umEmprestimo->p = umEmprestimo->p + (umEmprestimo->p * (umEmprestimo->j/100));
    else
        umEmprestimo->p = 0;
    (umEmprestimo->corrente)++;
    return retorno;
}

int main() {
    Emprestimo umEmprestimo;
    
    umEmprestimo.s = 200;
    umEmprestimo.n = 5;
    umEmprestimo.j = 1;
    
    novoEmprestimo(&umEmprestimo);
    
    int i = 1;
    float p = proximaParcela(&umEmprestimo);
    while (p > 0) {
        printf("O valor da parcela %d eh %3.2f\n", i, p);
        i++;
        p = proximaParcela(&umEmprestimo);
    }   
}

O valor da parcela 1 eh 200.00
O valor da parcela 2 eh 202.00
O valor da parcela 3 eh 204.02
O valor da parcela 4 eh 206.06
O valor da parcela 5 eh 208.12


## Exercício Parte 5 - Múltiplos Empréstimos

Considere que há múltiplos empréstimos que podem ser controlados em paralelo. A sua função `main` deve ser capaz de apresentar no console as parcelas de mais de um empréstimo de modo paralelo, mantendo um controle para cada empréstimo separadamente. Você deve decidir qual das soluções tomará como ponto de partida, se deve modificar a função `main`, a função `proximaParcela` ou ambas para atender esse requisito da melhor forma possível.

Adote uma solução compacta e generalizável, de tal modo que, cada novo empréstimo só exija a informação dos parâmetros de partida, sem expansão do código.

Por exemplo, suponha os seguintes dois empréstimos em paralelo:

### Empréstimo 1
* `S`: 200
* `N`: 5
* `J`: 1%

### Empréstimo 2
* `S`: 500
* `N`: 7
* `J`: 2%

A saída esperada é:
~~~
Emprestimo 1: parcela 1 eh 200.00
Emprestimo 2: parcela 1 eh 500.00
Emprestimo 1: parcela 2 eh 202.00
Emprestimo 2: parcela 2 eh 510.00
Emprestimo 1: parcela 3 eh 204.02
Emprestimo 2: parcela 3 eh 520.20
Emprestimo 1: parcela 4 eh 206.06
Emprestimo 2: parcela 4 eh 530.60
Emprestimo 1: parcela 5 eh 208.12
Emprestimo 2: parcela 5 eh 541.22
Emprestimo 2: parcela 6 eh 552.04
Emprestimo 2: parcela 7 eh 563.08
~~~

O exemplo mostra dois empréstimos, mas a estrutura deve ser genérica o suficiente para controlar N empréstimos em paralelo.

In [5]:
#include <stdio.h>

typedef struct {
    float s;
    int   n;
    float j;
    int   corrente;
    float p;
} Emprestimo;

Emprestimo novoEmprestimo(float s, int n, float j) {
    Emprestimo umEmprestimo;
    umEmprestimo.s = s;
    umEmprestimo.n = n;
    umEmprestimo.j = j;
    umEmprestimo.corrente = 1;
    umEmprestimo.p = s;
    return umEmprestimo;
}

float proximaParcela(Emprestimo *umEmprestimo) {
    float retorno = umEmprestimo->p;
    if (umEmprestimo->corrente < umEmprestimo->n)
        umEmprestimo->p = umEmprestimo->p + (umEmprestimo->p * (umEmprestimo->j/100));
    else
        umEmprestimo->p = 0;
    (umEmprestimo->corrente)++;
    return retorno;
}

int main() {
    Emprestimo emprestimo1,
               emprestimo2;
    
    emprestimo1 = novoEmprestimo(200, 5, 1);
    emprestimo2 = novoEmprestimo(500, 7, 2);
    
    float p1 = proximaParcela(&emprestimo1),
          p2 = proximaParcela(&emprestimo2);
    
    int i = 1;
    while (p1 > 0 || p2 > 0) {
        if (p1 > 0)
            printf("Emprestimo 1: parcela %d eh %3.2f\n", i, p1);
        if (p2 > 0)
            printf("Emprestimo 2: parcela %d eh %3.2f\n", i, p2);
        p1 = proximaParcela(&emprestimo1);
        p2 = proximaParcela(&emprestimo2);
        i++;
    }   
}

Emprestimo 1: parcela 1 eh 200.00
Emprestimo 2: parcela 1 eh 500.00
Emprestimo 1: parcela 2 eh 202.00
Emprestimo 2: parcela 2 eh 510.00
Emprestimo 1: parcela 3 eh 204.02
Emprestimo 2: parcela 3 eh 520.20
Emprestimo 1: parcela 4 eh 206.06
Emprestimo 2: parcela 4 eh 530.60
Emprestimo 1: parcela 5 eh 208.12
Emprestimo 2: parcela 5 eh 541.22
Emprestimo 2: parcela 6 eh 552.04
Emprestimo 2: parcela 7 eh 563.08
